In [26]:
import numpy as np
import pandas as pd
from preprocess import preprocess  # local file. restart kernel if this changed, it won't be re-imported otherwise
from sklearn.model_selection import train_test_split

## Read the first few rows during crude developing:
train = pd.read_csv('data/train.csv', nrows=1000).fillna(' ')
test = pd.read_csv('data/test.csv', nrows=1000).fillna(' ')

## These lines load all data:
#train = pd.read_csv('data/train.csv').fillna(' ')
#test = pd.read_csv('data/test.csv').fillna(' ')

[train, test, train_text, test_text, all_text, class_names] = preprocess(train, test)

# TODO why does a train dev split break the code?
# train, dev = train_test_split(train, test_size=0.1, random_state=42)

In [27]:
train.shape

(900, 8)

In [28]:
train.iloc[0:5]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
716,01f0ab0d0c384fb1,"DXRAW come out, come out, where ever you are\n...",0,0,0,0,0,0
351,00e0d3f34d4c0ce4,"Future Perfect at Sunrise|☼]] 14:59, 16",0,0,0,0,0,0
936,0293f0a89ba9b711,"""the proposed deletion process because of the...",0,0,0,0,0,0
256,00a330961879175c,"""*a short summary of referencing essentials\nF...",0,0,0,0,0,0
635,01b6cca55becc58b,"""\n\n Philosopher RFC \nphilosopher (from Wikt...",0,0,0,0,0,0


In [29]:
dev.iloc[0:5]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
521,015d1b0bb4cc744d,Dictionaries\n\nHow dare you call my contribut...,1,0,1,0,1,1
737,01fd0de4ca85352f,Tried to be productive (look at my contributio...,1,0,0,0,0,0
740,02004d4939cc3a95,If you agree with Master Bigode then what abou...,0,0,0,0,0,0
660,01cb9ca6df78b63d,"I told you I won't comment on it again, what m...",0,0,0,0,0,0
411,01076f023a99d133,"""\nEverywhere. Though I might just start going...",0,0,0,0,0,0


In [21]:
test.shape

(1000, 2)

In [2]:
import glove as glv
g_vectors, words_to_index, index_to_words = glv.loadGloveModel("data/glove.6B.50d.txt")

Loading Glove Model
Done. 400000  words loaded!


In [3]:
for v in words_to_index:
    if words_to_index[v] < 0:
        print("OMG")

In [4]:
g_vectors["hello"]

array([-0.38497 ,  0.80092 ,  0.064106, -0.28355 , -0.026759, -0.34532 ,
       -0.64253 , -0.11729 , -0.33257 ,  0.55243 , -0.087813,  0.9035  ,
        0.47102 ,  0.56657 ,  0.6985  , -0.35229 , -0.86542 ,  0.90573 ,
        0.03576 , -0.071705, -0.12327 ,  0.54923 ,  0.47005 ,  0.35572 ,
        1.2611  , -0.67581 , -0.94983 ,  0.68666 ,  0.3871  , -1.3492  ,
        0.63512 ,  0.46416 , -0.48814 ,  0.83827 , -0.9246  , -0.33722 ,
        0.53741 , -1.0616  , -0.081403, -0.67111 ,  0.30923 , -0.3923  ,
       -0.55002 , -0.68827 ,  0.58049 , -0.11626 ,  0.013139, -0.57654 ,
        0.048833,  0.67204 ])

In [5]:
max_len = train['comment_text'].str.split().apply(len).max()
max_len

1052

In [6]:
train[class_names].iloc[0]

toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
Name: 0, dtype: int64

In [7]:
train['comment_text'].iloc[0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [8]:
train['comment_text'].iloc[0].lower().split()

['explanation',
 'why',
 'the',
 'edits',
 'made',
 'under',
 'my',
 'username',
 'hardcore',
 'metallica',
 'fan',
 'were',
 'reverted?',
 'they',
 "weren't",
 'vandalisms,',
 'just',
 'closure',
 'on',
 'some',
 'gas',
 'after',
 'i',
 'voted',
 'at',
 'new',
 'york',
 'dolls',
 'fac.',
 'and',
 'please',
 "don't",
 'remove',
 'the',
 'template',
 'from',
 'the',
 'talk',
 'page',
 'since',
 "i'm",
 'retired',
 'now.89.205.38.27']

In [9]:
train['comment_text'].shape

(1000,)

In [10]:
from lstm_utils import sentences_to_indices, pretrained_embedding_layer

/usr/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, GRU, Activation
from keras.layers.embeddings import Embedding

def TheModel(input_shape, g_vectors, words_to_index):
    """
    Function creating the GRU model
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape = input_shape, dtype = 'int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(g_vectors, words_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)   
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    # X = LSTM(units = 128, return_sequences = True)(embeddings)
    X = GRU(units = 128)(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(rate = 0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    #X = LSTM(units = 128)(X)
    # Add dropout with a probability of 0.5
    #X = Dropout(rate = 0.5)(X)
    # Propagate X through a Dense (=FC) layer
    X = Dense(units = 6)(X)  # 6 units because 
    # Add a sigmoid activation because we have multilabel classification
    X = Activation("sigmoid", name = "sigmoid_activation")(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs = sentence_indices, outputs = X)
    
    return model

In [12]:
model = TheModel((max_len, ), g_vectors, words_to_index)

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1052)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1052, 50)          20000050  
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               68736     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 774       
_________________________________________________________________
sigmoid_activation (Activati (None, 6)                 0         
Total params: 20,069,560
Trainable params: 69,510
Non-trainable params: 20,000,050
___________________________________________________________

In [14]:
# https://github.com/keras-team/keras/issues/2166
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
Y_train = train.as_matrix(columns = class_names)

In [16]:
X_train_indices = sentences_to_indices(train['comment_text'], words_to_index, max_len)

In [17]:
print(X_train_indices.shape)
print(Y_train.shape)

(1000, 1052)
(1000, 6)


In [18]:
model.fit(X_train_indices, Y_train, epochs = 2, batch_size = 32, shuffle=True)

# Plot loss function during training:
# https://www.kaggle.com/parasjindal96/basic-deep-learning-tutorial-using-keras

Epoch 1/2
1000/1000 [==============================] - 30s 30ms/step - loss: 0.4181 - acc: 0.9612
Epoch 2/2
1000/1000 [==============================] - 35s 35ms/step - loss: 0.1822 - acc: 0.9585


In [19]:
max_len_dev = dev['comment_text'].str.split().apply(len).max()
X_dev_indices = sentences_to_indices(dev['comment_text'], words_to_index, max_len_dev)

Y_dev = dev.as_matrix(columns = class_names)

loss, acc = model.evaluate(X_dev_indices, Y_dev)
print()
print("Test accuracy = ", acc)

NameError: name 'dev' is not defined

In [ ]:
max_len_test = test['comment_text'].str.split().apply(len).max()
X_test_indices = sentences_to_indices(test['comment_text'], words_to_index, max_len_test)

Y_test = test.as_matrix(columns = class_names)

preds = model.predict(X_test_indices)
print()
print("Test accuracy = ", acc)

In [ ]:
# This code allows you to see the mislabelled examples
C = 5
y_test_oh = np.eye(C)[Y_test.reshape(-1)]
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_test_indices)
for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(num != Y_test[i]):
        print('Expected emoji:'+ label_to_emoji(Y_test[i]) + ' prediction: '+ X_test[i] + label_to_emoji(num).strip())